In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import tqdm

In [4]:
data_path = Path("/home/ET/mnwong/ML/data/Qobs10_SPCAMM.000.cam.h1.0001-02-13-00800.nc")
with xr.open_dataset(data_path) as ds:
    display(ds)
    print("\nData variables:", list(ds.data_vars))
    print("Coordinates:", list(ds.coords))

<xarray.Dataset>
Dimensions:       (lat: 384, lon: 576, lev: 30, ilev: 31, time: 27, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.53 -89.06 -88.59 ... 89.06 89.53 90.0
  * lon           (lon) float64 0.0 0.625 1.25 1.875 ... 357.5 358.1 358.8 359.4
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 0001-02-13 00:13:20 ... 0001-02-13 06:00:00
Dimensions without coordinates: nbnd
Data variables: (12/64)
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    hybi          (ilev) float64 ...
    ...            ...
    T             (time, lev, lat, lon) float64 ...
    TAUX          (time, lat, lon) float64 ...
    TAUY          (time, lat, lon) float64 ...
    U             (time, lev, lat, lon) float64 ...
    V             (time, lev, lat, lon) float64 ...
    Z3            (time, lev, lat, lon) float64 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              Qobs10_SPCAMM.000_cont
    logname:           tnchow
    host:              
    initial_file:      /home/ET/tnchow/cesm/Qobs10_SPCAMM.000/run/Qobs10_init...
    topography_file:   bnd_topo
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  second_800


Data variables: ['gw', 'hyam', 'hybm', 'P0', 'hyai', 'hybi', 'date', 'datesec', 'time_bnds', 'date_written', 'time_written', 'ndbase', 'nsbase', 'nbdate', 'nbsec', 'mdt', 'ndcur', 'nscur', 'co2vmr', 'ch4vmr', 'n2ovmr', 'f11vmr', 'f12vmr', 'sol_tsi', 'nsteph', 'CLDHGH', 'CLDICE', 'CLDLIQ', 'CLDLOW', 'CLDMED', 'CLDTOT', 'CLOUD', 'CLOUDTOP', 'DPRES', 'FLNS', 'FLNT', 'FSDS', 'FSNS', 'FSNT', 'HEIGHT', 'LHFLX', 'NUMICE', 'NUMLIQ', 'PHIS', 'PMID', 'PRECC', 'PRECSC', 'PS', 'Q', 'QRL', 'QRS', 'SHFLX', 'SPDQ', 'SPDQC', 'SPDQI', 'SPDT', 'SPNC', 'SPNI', 'T', 'TAUX', 'TAUY', 'U', 'V', 'Z3']
Coordinates: ['lat', 'lon', 'lev', 'ilev', 'time']


In [5]:
inputs_variable1 = ['U', 'V', 'T', 'Q', 'CLDLIQ', 'CLDICE', 'PMID', 'DPRES', 'Z3', 'HEIGHT']
inputs_variable2 = ['TAUX', 'TAUY', 'SHFLX', 'LHFLX']
output_variable1 = ['SPDQ', 'SPDQC', 'SPDQI', 'SPNC', 'SPNI', 'SPDT', 'CLOUD', 'CLOUDTOP']
output_variable2 = ['PRECC', 'PRECSC']

In [6]:
tensors_to_concat = []
for var in inputs_variable1:
    reshaped_tensor = torch.tensor(np.array(ds[var])).permute(0, 2, 3, 1).reshape(-1, ds[var].shape[1])
    mean = reshaped_tensor.mean(dim=0, keepdim=True)
    std = reshaped_tensor.std(dim=0, keepdim=True)
    reshaped_tensor = (reshaped_tensor - mean) / (std + 1e-6)
    tensors_to_concat.append(reshaped_tensor)
for var in inputs_variable2:
    reshaped_tensor = torch.tensor(np.array(ds[var])).reshape(-1, 1)
    mean = reshaped_tensor.mean(dim=0, keepdim=True)
    std = reshaped_tensor.std(dim=0, keepdim=True)
    reshaped_tensor = (reshaped_tensor - mean) / (std + 1e-6)
    tensors_to_concat.append(reshaped_tensor)

X = torch.cat(tensors_to_concat, dim=1)
X.shape

torch.Size([5971968, 305])

In [7]:
tensors_to_concat = []
for var in output_variable1:
    reshaped_tensor = torch.tensor(np.array(ds[var])).permute(0, 2, 3, 1).reshape(-1, ds[var].shape[1])
    mean = reshaped_tensor.mean(dim=0, keepdim=True)
    std = reshaped_tensor.std(dim=0, keepdim=True)
    reshaped_tensor = (reshaped_tensor - mean) / (std + 1e-6)
    tensors_to_concat.append(reshaped_tensor)
for var in output_variable2:
    reshaped_tensor = torch.tensor(np.array(ds[var])).reshape(-1, 1)
    mean = reshaped_tensor.mean(dim=0, keepdim=True)
    std = reshaped_tensor.std(dim=0, keepdim=True)
    reshaped_tensor = (reshaped_tensor - mean) / (std + 1e-6)
    tensors_to_concat.append(reshaped_tensor)

Y = torch.cat(tensors_to_concat, dim=1)
Y.shape

torch.Size([5971968, 242])

In [12]:
X = X.float()
Y = Y.float()

# 使用PyTorch进行数据集划分
torch.manual_seed(42)  # 设置随机种子以保证结果可重现

# 获取数据集大小
dataset_size = X.shape[0]
test_size = int(0.2 * dataset_size)  # 20%作为测试集
train_size = dataset_size - test_size

# 创建随机索引
indices = torch.randperm(dataset_size)
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# 根据索引分割数据
X_train = X[train_indices]
X_test = X[test_indices]
Y_train = Y[train_indices]
Y_test = Y[test_indices]

print(f"Training set - X: {X_train.shape}, Y: {Y_train.shape}")
print(f"Test set - X: {X_test.shape}, Y: {Y_test.shape}")

# 创建数据加载器
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

Training set - X: torch.Size([4777575, 305]), Y: torch.Size([4777575, 242])
Test set - X: torch.Size([1194393, 305]), Y: torch.Size([1194393, 242])


In [14]:
class NNCAM_CNN(nn.Module):
    def __init__(self):
        super(NNCAM_CNN, self).__init__()
        
        # 输入特征分组
        # 9个30维向量: indices 0-269 (9*30=270)
        # 1个31维向量: indices 270-300 (31)
        # 4个1维向量: indices 301-304 (4)
        
        # 为30维向量设计的1D CNN层
        self.conv1d_30 = nn.Sequential(
            nn.Conv1d(in_channels=9, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(15)  # 将30维降到15维
        )
        
        # 为31维向量设计的1D CNN层
        self.conv1d_31 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(15)  # 将31维降到15维
        )
        
        # 全连接层
        # 输入: 32*15 (来自30维) + 16*15 (来自31维) + 4 (1维向量) = 480 + 240 + 4 = 724
        self.fc_layers = nn.Sequential(
            nn.Linear(724, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 512)  # 准备解码
        )
        
        # 输出部分 - 反卷积
        # 8个30维向量的反卷积
        self.deconv1d_30 = nn.Sequential(
            nn.Linear(512, 8*15),  # 先映射到合适维度
            nn.ReLU()
        )
        
        self.deconv_30_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(30)  # 恢复到30维
        )
        
        # 2个1维向量的输出层
        self.output_1d = nn.Linear(512, 2)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # 分离不同类型的输入
        # 9个30维向量 (reshape为 batch_size, 9, 30)
        x_30d = x[:, :270].view(batch_size, 9, 30)
        
        # 1个31维向量 (reshape为 batch_size, 1, 31)
        x_31d = x[:, 270:301].view(batch_size, 1, 31)
        
        # 4个1维向量
        x_1d = x[:, 301:305]
        
        # 卷积处理
        conv_30 = self.conv1d_30(x_30d)  # (batch_size, 32, 15)
        conv_30 = conv_30.view(batch_size, -1)  # 展平
        
        conv_31 = self.conv1d_31(x_31d)  # (batch_size, 16, 15)
        conv_31 = conv_31.view(batch_size, -1)  # 展平
        
        # 连接所有特征
        combined = torch.cat([conv_30, conv_31, x_1d], dim=1)
        
        # 全连接层
        fc_out = self.fc_layers(combined)
        
        # 输出部分
        # 8个30维向量的输出
        deconv_input = self.deconv1d_30(fc_out)  # (batch_size, 8*15)
        deconv_input = deconv_input.view(batch_size, 8, 15)
        
        output_30d = self.deconv_30_layers(deconv_input)  # (batch_size, 8, 30)
        output_30d = output_30d.view(batch_size, -1)  # 展平为 (batch_size, 240)
        
        # 2个1维向量的输出
        output_1d = self.output_1d(fc_out)  # (batch_size, 2)
        
        # 连接所有输出
        output = torch.cat([output_30d, output_1d], dim=1)
        
        return output

# 创建模型实例
model = NNCAM_CNN()
print(f"Model created. Total parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# 检查设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = model.to(device)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

Model created. Total parameters: 701666
Using device: cuda


In [15]:
# 训练函数
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    num_batches = 0
    
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    return total_loss / num_batches

# 验证函数
def validate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            total_loss += loss.item()
            num_batches += 1
    
    return total_loss / num_batches

In [16]:
# 训练循环 - 使用tqdm可视化
try:
    from tqdm.notebook import tqdm
except ImportError:
    from tqdm import tqdm
import time

num_epochs = 10
train_losses = []
val_losses = []

print("开始训练CNN模型...")
start_time = time.time()

# 创建进度条
epoch_pbar = tqdm(range(num_epochs), desc="Training Progress")

for epoch in epoch_pbar:
    # 训练
    train_loss = train_model(model, train_loader, criterion, optimizer, device)
    
    # 验证
    val_loss = validate_model(model, test_loader, criterion, device)
    
    # 学习率调整
    scheduler.step()
    
    # 记录损失
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # 更新进度条
    epoch_pbar.set_postfix({
        'Train Loss': f'{train_loss:.6f}',
        'Val Loss': f'{val_loss:.6f}',
        'LR': f'{optimizer.param_groups[0]["lr"]:.2e}'
    })

total_time = time.time() - start_time
print(f"\n训练完成! 总用时: {total_time:.2f}秒")
print(f"最终训练损失: {train_losses[-1]:.6f}")
print(f"最终验证损失: {val_losses[-1]:.6f}")

开始训练CNN模型...


Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]


训练完成! 总用时: 2636.02秒
最终训练损失: 0.241669
最终验证损失: 0.259667


In [ ]:
# 可视化训练结果
plt.figure(figsize=(12, 5))

# 损失曲线
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, 'b-', label='训练损失', linewidth=2)
plt.plot(range(1, num_epochs + 1), val_losses, 'r-', label='验证损失', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('训练和验证损失曲线')
plt.legend()
plt.grid(True, alpha=0.3)

# 学习率变化
lr_values = []
for epoch in range(num_epochs):
    if epoch < 5:
        lr_values.append(0.001)
    else:
        lr_values.append(0.001 * (0.5 ** ((epoch - 4) // 5)))

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), lr_values, 'g-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('学习率变化')
plt.yscale('log')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 模型性能评估
print("\n=== 模型性能评估 ===")
print(f"最终训练损失: {train_losses[-1]:.6f}")
print(f"最终验证损失: {val_losses[-1]:.6f}")

# 计算损失的相对变化
if len(train_losses) > 1:
    train_improvement = ((train_losses[0] - train_losses[-1]) / train_losses[0]) * 100
    val_improvement = ((val_losses[0] - val_losses[-1]) / val_losses[0]) * 100
    print(f"训练损失改善: {train_improvement:.2f}%")
    print(f"验证损失改善: {val_improvement:.2f}%")

In [ ]:
# 测试集详细验证
def detailed_test_evaluation(model, test_loader, device):
    model.eval()
    predictions = []
    targets = []
    
    # 导入tqdm，支持不同环境
    try:
        from tqdm.notebook import tqdm
    except ImportError:
        from tqdm import tqdm
    
    print("在测试集上进行详细评估...")
    
    with torch.no_grad():
        test_pbar = tqdm(test_loader, desc="Testing")
        for batch_x, batch_y in test_pbar:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            outputs = model(batch_x)
            predictions.append(outputs.cpu())
            targets.append(batch_y.cpu())
            
            # 计算当前批次的MSE
            batch_mse = criterion(outputs, batch_y).item()
            test_pbar.set_postfix({'MSE': f'{batch_mse:.6f}'})
    
    # 合并所有预测和目标
    predictions = torch.cat(predictions, dim=0)
    targets = torch.cat(targets, dim=0)
    
    # 计算各种评估指标
    mse = nn.MSELoss()(predictions, targets).item()
    rmse = np.sqrt(mse)
    mae = nn.L1Loss()(predictions, targets).item()
    
    # 计算R²分数
    targets_mean = targets.mean()
    ss_tot = ((targets - targets_mean) ** 2).sum()
    ss_res = ((targets - predictions) ** 2).sum()
    r2_score = 1 - (ss_res / ss_tot)
    
    print(f"\n=== 测试集评估结果 ===")
    print(f"均方误差 (MSE): {mse:.6f}")
    print(f"均方根误差 (RMSE): {rmse:.6f}")
    print(f"平均绝对误差 (MAE): {mae:.6f}")
    print(f"R²分数: {r2_score:.6f}")
    
    return predictions, targets, mse, rmse, mae, r2_score

# 执行详细测试
predictions, targets, test_mse, test_rmse, test_mae, r2 = detailed_test_evaluation(model, test_loader, device)

In [ ]:
# 预测结果可视化
plt.figure(figsize=(15, 10))

# 选择一些样本进行可视化
n_samples = min(5, predictions.shape[0])
sample_indices = np.random.choice(predictions.shape[0], n_samples, replace=False)

for i, idx in enumerate(sample_indices):
    plt.subplot(2, 3, i+1)
    
    pred = predictions[idx].numpy()
    target = targets[idx].numpy()
    
    plt.plot(target, 'b-', label='真实值', linewidth=2, alpha=0.8)
    plt.plot(pred, 'r--', label='预测值', linewidth=2, alpha=0.8)
    plt.xlabel('输出维度')
    plt.ylabel('值')
    plt.title(f'样本 {idx+1}')
    plt.legend()
    plt.grid(True, alpha=0.3)

# 整体相关性图
plt.subplot(2, 3, 6)
pred_flat = predictions.flatten().numpy()
target_flat = targets.flatten().numpy()

# 随机采样以避免图片过于密集
sample_size = min(10000, len(pred_flat))
sample_idx = np.random.choice(len(pred_flat), sample_size, replace=False)

plt.scatter(target_flat[sample_idx], pred_flat[sample_idx], alpha=0.5, s=1)
plt.plot([target_flat.min(), target_flat.max()], [target_flat.min(), target_flat.max()], 'r--', linewidth=2)
plt.xlabel('真实值')
plt.ylabel('预测值')
plt.title(f'预测 vs 真实值\nR² = {r2:.4f}')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== CNN模型架构总结 ===")
print(f"输入特征:")
print(f"  - 9个30维向量 (通过1D CNN处理)")
print(f"  - 1个31维向量 (通过1D CNN处理)") 
print(f"  - 4个1维向量 (直接连接)")
print(f"\n输出特征:")
print(f"  - 8个30维向量 (通过反卷积生成)")
print(f"  - 2个1维向量 (通过全连接层生成)")
print(f"\n模型参数总数: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"训练样本数: {X_train.shape[0]:,}")
print(f"测试样本数: {X_test.shape[0]:,}")
print(f"批次大小: 256")
print(f"训练轮数: 10")